In [ ]:
import logging
logging.basicConfig()
logger = logging.getLogger()
# logger.setLevel(logging.INFO)

from models.analytics.stats_group import StatsGroupAnalytics
from models.analytics.base_conditions_group import BaseConditionsGroupAnalytics
from models.analytics.agent_base_conditions import AgentBaseConditionsAnalytics
from definitions import HISTORICAL_MANAGER_FOLDER, BASE_CONDITIONS_FOLDER
from graphs import precision_recall_group_graph, score_group_graph, heatmap

seeds = ['s1', 's2','s3', 's4', 's5' ]
groups = ['sp10', 'sp50', 'sp100', 'sp200']

In [ ]:
stats_analytics= StatsGroupAnalytics(HISTORICAL_MANAGER_FOLDER, seeds, groups)
summary = stats_analytics.get_summary()

In [ ]:
import seaborn as sns

sns.color_palette("mako", as_cmap=True)

score_group_graph(groups, summary, seeds)

metrics_columns = ['specialist_tp', 'specialist_fp', 'specialist_fn', 'specialist_tn']
metrics = stats_analytics.get_stats(metrics_columns)

precision_recall_group_graph(metrics, groups, seeds)

In [ ]:
import pandas as pd

specialists_heat = {}

for group in groups:
    base_conditions_analytics = BaseConditionsGroupAnalytics(HISTORICAL_MANAGER_FOLDER, seeds, [group])
    specialist_heatmap_list = base_conditions_analytics.get_summary()

    df = pd.DataFrame(specialist_heatmap_list[0])
    specialist_heat_df = df * 1000

    specialists_heat[group] = specialist_heat_df
    heatmap(specialist_heat_df.T, title=f'Specialist Performance Through Conditions X Generation {seeds} [group {group}]')

In [ ]:
agent_conditions = AgentBaseConditionsAnalytics(BASE_CONDITIONS_FOLDER, seeds)

for seed in seeds:
    df = agent_conditions.get_seed(seed)
    heatmap(df.data.T, title=f'Agent Performance Through Conditions X Generation {seed}')

agent_heat_df = agent_conditions.get_mean()

heatmap(agent_heat_df.T, title=f'Agent Performance Through Conditions X Generation {seeds} Mean')

In [ ]:
start = len(agent_heat_df) - len(specialist_heat_df)

agent_heat_df = agent_heat_df[start:].reset_index()
diff = agent_heat_df.subtract(specialist_heat_df)

heatmap(diff.T.abs(), title=f'Difference of Agent/Specialist Performance Through Conditions X Generation {seeds}', colorscale='temps')